# RNA - Atividade 2.1 - Perceptron de Rosenblatt

Neste projeto prático, o objetivo é implementar o algoritmo de treinamento mediante Aprendizado Supervisionado do neurônio Perceptron de Rosenblatt aplicado em problemas de classificação. Para tanto, cada equipe deverá elaborar Jupyter Notebooks com o código-fonte deste algoritmo de treinamento desenvolvido na linguagem de Programação Python e fazendo uso das bibliotecas numpy, random, math e matplotlib. Em particular, a biblioteca numpy será de uso obrigatório para todas as operações de natureza matricial (multiplicação de matrizes, produto escalar, etc). Neste projeto prático, a biblioteca sci-kit learn só deve ser utilizada para o cálculo de métricas de desempenho.

## Equipe

- Daniele Simas - 2015310060
- Felipe Amorim - 2115080033
- José Manuel - 2115080052
- Miguel Angelo - 2115080024

## Importação de Bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Parte 1 - Resolvendo um problema linearmente separável

Nesta parte, todas as equipes devem usar o arquivo `dataAll.txt` e construir o algoritmo de treinamento do neurônio perceptron para resolver o problema de classificação proposto.

In [18]:
# Classe para controlar o treinamento do perceptron
from src.perceptron import Perceptron

# Funções auxiliares para carregar os arquivos do exercicio
from src.utils import read_data, identifier

# Funções de ativação
import src.activation_functions

In [12]:
data = read_data('All')

print("Quantidade de pontos lidos: ", len(data))
print("Formato dos dados: ", data.shape)
print("\nExemplos de dados: ", *[(x, y) for x,y in data[:5]], sep='\n')

Quantidade de pontos lidos:  1000
Formato dos dados:  (1000, 2)

Exemplos de dados: 
(array([-363.7884,  244.1423]), 0.0)
(array([ 328.7572, -256.7658]), 1.0)
(array([-164.9827,  613.2164]), 0.0)
(array([536.5905, 764.9575]), 0.0)
(array([314.2297, 343.8375]), 0.0)


1. As equipes devem utilizar a função de ativação degrau com ϑ = 0;
2. O valor da taxa de aprendizado deve ser igual a η = 0,1;
3. O vetor inicial de pesos deve ter seus valores inicializados conforme uma variável aleatória de distribuição uniforme no intervalo, isto é, wi ∼ U (−0,5, + 0,5). O vetor inicial de pesos deve ser impresso no início da execução do algoritmo;

In [39]:
perceptron = Perceptron(data=data, activation_function=activation_functions.step_function, bias=-1, learning_rate=0.1)

print('Vetor de pesos inicial: ', perceptron.weights)

perceptron.randomize_weights(floor=-0.5, ceiling=0.501)
print('Vetor de pesos após a randomização: ', perceptron.weights)

Vetor de pesos inicial:  [0. 0. 0.]
Vetor de pesos após a randomização:  [-0.11215026 -0.26412487  0.4925133 ]


4. A cada época deve ser indicado o número de ajustes feitos no vetor de pesos;
5. O algoritmo deve executar até a convergência, isto é, até que não haja erros para todos os exemplos presentes no conjunto de treinamento;
6. Ao final, deve-se imprimir:
    - A - O número total de ajustes no vetor de pesos;
    - B - O número de épocas até a convergência;
    - C - O gráfico contendo todos os exemplos do conjunto de dados e a reta que separa as classes obtida como resultado do treinamento do neurônio Perceptron. Respeitar o esquema de cores proposto inicialmente e apresentar a solução de maneira clara neste gráfico.

In [40]:
epoch, updates = perceptron.train()

print('Treinamento finalizado!')
print('Quantidade de épocas até a convergência: ', epoch)
print('Quantidade de ajustes no vetor de pesos: ', updates)

print("Vetor de pesos final: ", perceptron.weights)

Treinamento finalizado!
Quantidade de épocas até a convergência:  4
Quantidade de ajustes no vetor de pesos:  55
Vetor de pesos final:  [  -0.61215026  323.14490513 -323.0183267 ]


In [41]:
def plot_results(data, final_weights):
    # INCLUIR AQUI A LÓGICA DE CRIAÇÃO DO GRÁFICO
    plt.figure(figsize=(8,8))
    
    # Criar gráfico de dispersão dos pontos de entrada usando o parametro "data"
    # plt.scatter
    

    # Criar gráfico de linha da reta x2 = -(w1/w2)x1 + (w0/x1) gerada pelo perceptron
    # plt.plot
    w0, w1, w2 = final_weights
    x2 = lambda x1: (w0/w2) - (w1/w2)*x1